# Знакомство с нейронными сетями (решение задач с MNIST)

## Задание

Поробуйте изменить модель и процесс обучения в соответсвии с разделами "Эксперименты с..". Сделайте вывод о том, какие эксперименты были удачными

Читать здесь: [Эксперименты с многослойным перцептроном в Keras](http://deep.uran.ru/wiki/index.php?title=%D0%AD%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D0%B8%D0%BC%D0%B5%D0%BD%D1%82%D1%8B_%D1%81_%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D1%8B%D0%BC_%D0%BF%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD%D0%BE%D0%BC_%D0%B2_Keras).

In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

# Вспомогательные библиотеки
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Подключение Keras

In [ ]:
from keras.datasets import mnist

## Загрузка обучающих и тестовых примеров из MNIST

In [ ]:
nb_classes = 10

# the data, shuffled and split between tran and test sets
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Давайте посмотрим на формат данных перед обучением модели. Воспользовавшись `shape` мы видим, что в тренировочном датасете 60,000 изображений, каждое размером 28 x 28 пикселей:

In [ ]:
train_images.shape

In [ ]:
train_labels

## Предобработка данных



Данные должны быть предобработаны перед обучением нейросети. Если вы посмотрите на первое изображение в тренировочном сете вы увидите, что значения пикселей находятся в диапазоне от 0 до 255:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Мы масштабируем эти значения к диапазону от 0 до 1 перед тем как скормить их нейросети. Для этого мы поделим значения на 255. Важно, чтобы тренировочный сет и проверочный сет были предобработаны одинаково:

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
class_names = np.arange(10)
class_names

Чтобы убедиться, что данные в правильном формате и мы готовы построить и обучить нейросеть, выведем на экран первые 25 изображений из тренировочного сета и отобразим под ними наименования их классов.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Создание сети - однослойного перцептрона

### Описание модели



Создадим однослойный перцептрон в Keras. Входные примеры базы MNIST являются изображениями 28x28 пикселей, которые преобразованы в вектора длины 784 через слой `Flatten` (его мы не будем считать как за слой, он для нас лишь входной вектор). Мы создаем слой, состоящий из 10 нейронов, в котором каждый нейрон связян со всеми входами. Такие слои в Keras называются "Dense". Мы задаём функцию активации выходного слоя - `softmax`, которая применяется для задач классификации. То есть, на выходе будет выдаваться 10 неотрицательных чисел, сумма которых равна 1, которые характеризуют вероятности того, что входное изображение является цифрой 0,1,...9.



In [ ]:
model1 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(10, activation='softmax')
])

model1.summary() #Print model info

### Компиляция модели



Теперь, скомпилируем модель. Это значит, что мы построим функцию Python, которая позволит вычислять результат работы сети на входном векторе, вызывая функцию `model.predict(...)`. Но, главное, посчитается функция вычисления градиента функции ошибки по весам сети, что необходимо для осуществления обучения (настройки) параметров сети:

In [ ]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Здесь `adam` - тип градиентного спуска, `categorical_crossentropy` - функция штрафа, кроссэнтропия, которую следует использовать для задач классификации, как у нас, `metrics=['accuracy']` значит, что мы будем вычислять в модели не только функцию штрафа, но и точность работы, то есть, число правильно классифицированных примеров.

### Обучение модели



Это самая ресурсоемкая операция. Мы выполняем обучение "пачками" (mini-batch) по `batch_size=128 примеров`. При этом, осуществляет `epoch=5` проходов по всем входным примерам.

Другие варианты режима обучения:

* **по одному примеру** - медленная сходимость, и алгоритм "не видит" хороших минимумов,
* **по всем примерам сразу** - трудоемко, и сходимость к локальному минимуму и склонностьк переобучению.

А обработка `mini-batch` - промежуточный вариант, который называется градиентным стохастическим спуском.

In [ ]:
model1.fit(train_images, train_labels, batch_size=128, verbose=2, epochs=10, validation_data=(test_images, test_labels))

Здесь `loss` - функция ошибки, `acc` - точность на обучающей выборке, `val_loss` и `val_acc` - на тестовой выборке.

## Эксперименты с однослойной сетью

### Увеличение числа прогонов

Измените количество прогонов обучающих до 20. Изменится ли что-нибудь при  увеличении числа прогонов?

### Выключение нормализации входных данных

Закомментируем нормализацию входных яркостей:

```python
# train_images = train_images / 255.0
# test_images = test_images / 255.0
```

Как сказывается это на процесс обучения?

### Верните все в исходное состояние!



Прежде чем двигаться дальше, раскомментируйте нормализацию входных яркостей обратно и перезапустите блокнот `Runtime -> Restart and run all...`:

```python
train_images = train_images / 255.0
test_images = test_images / 255.0
```

## Двуслойный перцептрон



Создадим теперь сеть с двумя "плотными" слоями.

Первый слой ("скрытый слой") будет состоять из 100 нейронов, соединенных со всеми входами. Функция активации будет `relu` - это самая популярная функция активации для скрытых слоев.
Второй слой будет состоять из 10 нейронов с функцией активации `softmax`.

In [ ]:
model2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model2.summary() #Print model info

In [ ]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(train_images, train_labels, batch_size=128, verbose=2, epochs=10, validation_data=(test_images, test_labels))

В этом случае, у сети будет уже 79510 параметров (в однослойном случае было 7850).

Из теоремы Колмогорова следует, *что любую функцию можно аппроксимировать с любой точностью, используя двуслойную сеть, задав достаточно много нейронов в первом слое*. Правда, эта теорема не говорит о том, что обучение сети сойдется к нужной аппроксимации, но указывает, что теоретически, достаточно двух слоев.

 

## Эксперименты с двухслойной сетью

### Изменение количество нейронов в слое



Попробуйте поиграться с количеством нейронов в скрытом слое. Изменится ли точность на тестовой выборке?

## Трехслойный перцептрон

Добавьте еще один скрытый слой из 200 нейронов после первого слоя двуслойной сети.

## Эксперименты с двухслойной сетью

### Изменение количество нейронов в слое

Попробуйте поиграться с количеством нейронов в скрытом слое. Изменится ли точность на тестовой выборке?

### Изменение функций активации



Попробуйте изменить функцию активации в скрытых слоях `sigmoid` или `tanh` Подробнее о функциях активации см. [здесь](https://keras.io/activations/). Как это скажется на процессе обучения, валидации?



## Выводы

Мы получили следующие результаты:

* **Однослойная сеть**: `/вставь сюда что получилось/` параметров, точность `/вставь сюда что получилось/`%
* **Двуслойная сеть**: `/вставь сюда что получилось/` параметров, точность `/вставь сюда что получилось/`%
* **Трехслойная сеть**: `/вставь сюда что получилось/` параметров, точность `/вставь сюда что получилось/`%


Из таблицы следует, что двуслойная "плотная" сеть работает  `/лучше/худше/также/` однослойной, но трехслойная сеть работает `/лучше/худше/также/` двухслойной.

Может возникнуть эффект переобучения от увеличение количества слоев?